## Домашнее задание №1 "Ключевые слова"

### Настя Шабаева, БКЛ181

Корпус для этого задания был собран из статей онлайн версиии журнала "Вокруг света" (https://www.vokrugsveta.ru), из рубрики #Истории. Такого понятия как "ключевые слова" в журнале не выделяется, каждой статье в журнале присваиваются определенные теги. Они обозначаются в отдельном микро-разделе после каждой статьи.

![tags](./tags.png)

In [1]:
import pandas as pd

from string import punctuation
punctuation += '...' + '—' + '…' + '«»'

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('russian')
stop.extend(["который", "это"])

from pymorphy2 import MorphAnalyzer
from pymorphy2.tokenizers import simple_word_tokenize
MorphAn = MorphAnalyzer()

import RAKE
rake = RAKE.Rake(stop)

from summa import keywords

from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(stop_words=stop, smooth_idf=True, 
                             use_idf=True, ngram_range=(1, 3))

[nltk_data] Downloading package stopwords to /Library/Frameworks/Pytho
[nltk_data]     n.framework/Versions/3.7/lib/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
dataframe = pd.read_csv("texts_keywords.csv")

In [3]:
dataframe

,number,title,text,tags,my_keywords,standart_keywords
0,1,Юбилей: выпускайте кракена,Ровно 160 лет назад состоялась встреча человек...,"Юбилей, Кальмар, Ноябрь 2021, Лучшее","Монстр, Гигантский кальмар, Глубины моря, Живо...","Монстр, Гигантский кальмар, Глубины моря, Живо..."
1,2,"Свет, цвет, движение: секреты макросъемки",Читатели журнала и сайта «Вокруг света» наверн...,"Истории, Фотография, Лайфхаки","Макросъемка, Оборудование, Макрофотоохота, При...","Фотография, Макросъемка, Оборудование, Макрофо..."
2,3,"Мякоть субъективности: 10 тайных символов, скр...",Первая галлюцинация случилась у дали в детстве...,"Сальвадор Дали, Живопись, Искусство, Художник,...","Мягкие часы, Пространство, Время, Вдохновение,...","Сальвадор Дали, Искусство, Художник, Картина, ..."
3,4,Железное рукопожатие,"Порой небольшие детали — почтовые ящики, дорож...","Истории, Путешествия, Марокко","Декор, Дом, Дверь, Ручка, Талисман, Традиция","Марокко, Декор, Дом, Дверь, Ручка, Талисман, Т..."
4,5,«Помню не зря пятый день ноября»: как недоволь...,В ночь на 5 ноября в Великобритании отмечается...,"Англия, Католицизм, Монархия, Истории","Протестантизм, Католик, Протестант, Король, Па...","Англия, Католицизм, Монархия, Протестантизм, К..."


В колонке tags перечислены теги, которыми размечены статьи в журнале "Вокруг света", в колонке my_keywords - ключевые слова, выделенные мной самостоятельно, в колонке standart_keywords - в какой-то мере, объединение двух предыдущих колонок, только из колонки tags были удалены теги, которые не встречаются в самом тексте статей.

Стоит отметить, что теги, которыми размечены статьи в источнике, часто оказываются довольно общими и размытыми, поэтому при составлении своей разметки я старалась делать акцент на большей конкретике ключевых слов. В связи с этим, разметка источника и моя разметка не сильно пересекались.

### Предобработка текстов и ключевых слов

In [4]:
dataframe["standart_keywords_list"] = [cell.lower().split(', ') 
                                       for cell in dataframe["standart_keywords"]] 

Лемматизируем эталонныхе ключевые слова.

In [5]:
standart_kwords_list = []
for kwords in dataframe["standart_keywords_list"]:
    kwords_list = []
    for kword in kwords:
        help_list = []
        for word in kword.split(" "):
            help_list.append(MorphAn.parse(word)[0].normal_form)
        kwords_list.append(" ".join(help_list))
    standart_kwords_list.append(kwords_list)
dataframe["standart_keywords_list"] = standart_kwords_list

In [6]:
dataframe

,number,title,text,tags,my_keywords,standart_keywords,standart_keywords_list
0,1,Юбилей: выпускайте кракена,Ровно 160 лет назад состоялась встреча человек...,"Юбилей, Кальмар, Ноябрь 2021, Лучшее","Монстр, Гигантский кальмар, Глубины моря, Живо...","Монстр, Гигантский кальмар, Глубины моря, Живо...","[монстр, гигантский кальмар, глубина море, жив..."
1,2,"Свет, цвет, движение: секреты макросъемки",Читатели журнала и сайта «Вокруг света» наверн...,"Истории, Фотография, Лайфхаки","Макросъемка, Оборудование, Макрофотоохота, При...","Фотография, Макросъемка, Оборудование, Макрофо...","[фотография, макросъемка, оборудование, макроф..."
2,3,"Мякоть субъективности: 10 тайных символов, скр...",Первая галлюцинация случилась у дали в детстве...,"Сальвадор Дали, Живопись, Искусство, Художник,...","Мягкие часы, Пространство, Время, Вдохновение,...","Сальвадор Дали, Искусство, Художник, Картина, ...","[сальвадор дать, искусство, художник, картина,..."
3,4,Железное рукопожатие,"Порой небольшие детали — почтовые ящики, дорож...","Истории, Путешествия, Марокко","Декор, Дом, Дверь, Ручка, Талисман, Традиция","Марокко, Декор, Дом, Дверь, Ручка, Талисман, Т...","[марокко, декор, дом, дверь, ручка, талисман, ..."
4,5,«Помню не зря пятый день ноября»: как недоволь...,В ночь на 5 ноября в Великобритании отмечается...,"Англия, Католицизм, Монархия, Истории","Протестантизм, Католик, Протестант, Король, Па...","Англия, Католицизм, Монархия, Протестантизм, К...","[англия, католицизм, монархия, протестантизм, ..."


In [7]:
lemmatized_text = []
for index, text in enumerate(dataframe["text"]):
    lemmas = []
    for t in simple_word_tokenize(text):
        #if t not in punctuation: - хотела убрать пунктуацию из текстов, но при этом RAKE временами перестает работать
        lemmas.append(
            MorphAn.parse(t)[0].normal_form
        )
    lemmatized_text.append(' '.join(lemmas))

dataframe["lemmatized_text"] = lemmatized_text

В колонке lemmatized_text склеенный лемматизированный текст, в колонке standart_keywords_list ключевые слова списком в нижнем регистре

In [8]:
dataframe

,number,title,text,tags,my_keywords,standart_keywords,standart_keywords_list,lemmatized_text
0,1,Юбилей: выпускайте кракена,Ровно 160 лет назад состоялась встреча человек...,"Юбилей, Кальмар, Ноябрь 2021, Лучшее","Монстр, Гигантский кальмар, Глубины моря, Живо...","Монстр, Гигантский кальмар, Глубины моря, Живо...","[монстр, гигантский кальмар, глубина море, жив...",ровно 160 год назад состояться встреча человек...
1,2,"Свет, цвет, движение: секреты макросъемки",Читатели журнала и сайта «Вокруг света» наверн...,"Истории, Фотография, Лайфхаки","Макросъемка, Оборудование, Макрофотоохота, При...","Фотография, Макросъемка, Оборудование, Макрофо...","[фотография, макросъемка, оборудование, макроф...",читатель журнал и сайт « вокруг свет » наверня...
2,3,"Мякоть субъективности: 10 тайных символов, скр...",Первая галлюцинация случилась у дали в детстве...,"Сальвадор Дали, Живопись, Искусство, Художник,...","Мягкие часы, Пространство, Время, Вдохновение,...","Сальвадор Дали, Искусство, Художник, Картина, ...","[сальвадор дать, искусство, художник, картина,...",один галлюцинация случиться у дать в детство ....
3,4,Железное рукопожатие,"Порой небольшие детали — почтовые ящики, дорож...","Истории, Путешествия, Марокко","Декор, Дом, Дверь, Ручка, Талисман, Традиция","Марокко, Декор, Дом, Дверь, Ручка, Талисман, Т...","[марокко, декор, дом, дверь, ручка, талисман, ...","порой небольшой деталь — почтовый ящик , дорож..."
4,5,«Помню не зря пятый день ноября»: как недоволь...,В ночь на 5 ноября в Великобритании отмечается...,"Англия, Католицизм, Монархия, Истории","Протестантизм, Католик, Протестант, Король, Па...","Англия, Католицизм, Монархия, Протестантизм, К...","[англия, католицизм, монархия, протестантизм, ...",в ночь на 5 ноябрь в великобритания отмечаться...


### RAKE

In [9]:
dataframe["RAKE_keywords"] = [rake.run(text, minFrequency=2, maxWords=3) 
                              for text in list(dataframe["lemmatized_text"])]

Убираем те ключевые слова, ratio у которых 0.

In [10]:
rake_kwords_list = []
for kwords in dataframe["RAKE_keywords"]:
    kwords_list = []
    for word in kwords:
        if word[1] != 0:
            kwords_list.append(word[0])
    rake_kwords_list.append(kwords_list)
dataframe["RAKE_keywords"] = rake_kwords_list

In [11]:
dataframe

,number,title,text,tags,my_keywords,standart_keywords,standart_keywords_list,lemmatized_text,RAKE_keywords
0,1,Юбилей: выпускайте кракена,Ровно 160 лет назад состоялась встреча человек...,"Юбилей, Кальмар, Ноябрь 2021, Лучшее","Монстр, Гигантский кальмар, Глубины моря, Живо...","Монстр, Гигантский кальмар, Глубины моря, Живо...","[монстр, гигантский кальмар, глубина море, жив...",ровно 160 год назад состояться встреча человек...,"[бросить вызов наука, легенда, доказать, наход..."
1,2,"Свет, цвет, движение: секреты макросъемки",Читатели журнала и сайта «Вокруг света» наверн...,"Истории, Фотография, Лайфхаки","Макросъемка, Оборудование, Макрофотоохота, При...","Фотография, Макросъемка, Оборудование, Макрофо...","[фотография, макросъемка, оборудование, макроф...",читатель журнал и сайт « вокруг свет » наверня...,"[объект съёмка, объектив, растение, пример, 10..."
2,3,"Мякоть субъективности: 10 тайных символов, скр...",Первая галлюцинация случилась у дали в детстве...,"Сальвадор Дали, Живопись, Искусство, Художник,...","Мягкие часы, Пространство, Время, Вдохновение,...","Сальвадор Дали, Искусство, Художник, Картина, ...","[сальвадор дать, искусство, художник, картина,...",один галлюцинация случиться у дать в детство ....,"[слово нина геташвили, самый знаменитый, худож..."
3,4,Железное рукопожатие,"Порой небольшие детали — почтовые ящики, дорож...","Истории, Путешествия, Марокко","Декор, Дом, Дверь, Ручка, Талисман, Традиция","Марокко, Декор, Дом, Дверь, Ручка, Талисман, Т...","[марокко, декор, дом, дверь, ручка, талисман, ...","порой небольшой деталь — почтовый ящик , дорож...","[злой дух, пятиконечный звезда, ручка, спасать..."
4,5,«Помню не зря пятый день ноября»: как недоволь...,В ночь на 5 ноября в Великобритании отмечается...,"Англия, Католицизм, Монархия, Истории","Протестантизм, Католик, Протестант, Король, Па...","Англия, Католицизм, Монархия, Протестантизм, К...","[англия, католицизм, монархия, протестантизм, ...",в ночь на 5 ноябрь в великобритания отмечаться...,"[католический церковь, судьбоносный день, церк..."


##### Ключевые слова, предсказанные алгоритмом RAKE

In [12]:
for index, text in enumerate(dataframe["RAKE_keywords"]):
    print(list(dataframe["title"])[index])
    print("RAKE: ")
    print(text)
    print("Standart: ")
    print(list(dataframe["standart_keywords_list"])[index])
    print("\n")

Юбилей: выпускайте кракена
RAKE: 
['бросить вызов наука', 'легенда', 'доказать', 'находиться', 'вода', 'существование', 'относиться', 'кракен']
Standart: 
['монстр', 'гигантский кальмар', 'глубина море', 'животное', 'морской легенда', 'миф', 'щупальце', 'кракен', 'кальмар']


Свет, цвет, движение: секреты макросъемки
RAKE: 
['объект съёмка', 'объектив', 'растение', 'пример', '100 мм', 'макрокольцо', 'кроме', 'ножницы']
Standart: 
['фотография', 'макросъемка', 'оборудование', 'макрофотоохота', 'природа', 'знание', 'насекомое', 'техника', 'животное', 'объектив', 'снимка']


Мякоть субъективности: 10 тайных символов, скрытых в самой известной картине Сальвадора Дали
RAKE: 
['слово нина геташвили', 'самый знаменитый', 'художник', 'дать', 'картина', 'картина —', 'сон', 'дело', '— прима', 'ред', 'беззащитность']
Standart: 
['сальвадор дать', 'искусство', 'художник', 'картина', 'мягкий часы', 'пространство', 'время', 'вдохновение', 'часы', 'сон', 'реальность']


Железное рукопожатие
RAKE: 
['

### TextRank

In [13]:
dataframe["TextRank_keywords"] = [keywords.keywords(text, 
                                                    language='russian', 
                                                    additional_stopwords=stop, 
                                                    scores=True) 
                                  for text in list(dataframe["lemmatized_text"])]

In [14]:
trank_kwords_list = []
for kwords in dataframe["TextRank_keywords"]:
    kwords_list = []
    for word in kwords:
        if word[1] != 0:
            kwords_list.append(word[0])
    trank_kwords_list.append(kwords_list)
dataframe["TextRank_keywords"] = trank_kwords_list

In [15]:
dataframe

,number,title,text,tags,my_keywords,standart_keywords,standart_keywords_list,lemmatized_text,RAKE_keywords,TextRank_keywords
0,1,Юбилей: выпускайте кракена,Ровно 160 лет назад состоялась встреча человек...,"Юбилей, Кальмар, Ноябрь 2021, Лучшее","Монстр, Гигантский кальмар, Глубины моря, Живо...","Монстр, Гигантский кальмар, Глубины моря, Живо...","[монстр, гигантский кальмар, глубина море, жив...",ровно 160 год назад состояться встреча человек...,"[бросить вызов наука, легенда, доказать, наход...","[животное, животный, гигантский, капитан, буйе..."
1,2,"Свет, цвет, движение: секреты макросъемки",Читатели журнала и сайта «Вокруг света» наверн...,"Истории, Фотография, Лайфхаки","Макросъемка, Оборудование, Макрофотоохота, При...","Фотография, Макросъемка, Оборудование, Макрофо...","[фотография, макросъемка, оборудование, макроф...",читатель журнал и сайт « вокруг свет » наверня...,"[объект съёмка, объектив, растение, пример, 10...","[фотография, фотограф, макросъемка, большой, ф..."
2,3,"Мякоть субъективности: 10 тайных символов, скр...",Первая галлюцинация случилась у дали в детстве...,"Сальвадор Дали, Живопись, Искусство, Художник,...","Мягкие часы, Пространство, Время, Вдохновение,...","Сальвадор Дали, Искусство, Художник, Картина, ...","[сальвадор дать, искусство, художник, картина,...",один галлюцинация случиться у дать в детство ....,"[слово нина геташвили, самый знаменитый, худож...","[дать, художник стать, время, символ, час, поэ..."
3,4,Железное рукопожатие,"Порой небольшие детали — почтовые ящики, дорож...","Истории, Путешествия, Марокко","Декор, Дом, Дверь, Ручка, Талисман, Традиция","Марокко, Декор, Дом, Дверь, Ручка, Талисман, Т...","[марокко, декор, дом, дверь, ручка, талисман, ...","порой небольшой деталь — почтовый ящик , дорож...","[злой дух, пятиконечный звезда, ручка, спасать...","[город, марокко ручка дом, страна, странно, си..."
4,5,«Помню не зря пятый день ноября»: как недоволь...,В ночь на 5 ноября в Великобритании отмечается...,"Англия, Католицизм, Монархия, Истории","Протестантизм, Католик, Протестант, Король, Па...","Англия, Католицизм, Монархия, Протестантизм, К...","[англия, католицизм, монархия, протестантизм, ...",в ночь на 5 ноябрь в великобритания отмечаться...,"[католический церковь, судьбоносный день, церк...","[свой, год, годами, весь, фокс, фоксом, самый,..."


##### Ключевые слова, предсказанные алгоритмом TextRank

In [16]:
for index, text in enumerate(dataframe["TextRank_keywords"]):
    print(list(dataframe["title"])[index])
    print("TextRank: ")
    print(text)
    print("Standart: ")
    print(list(dataframe["standart_keywords_list"])[index])
    print("\n")

Юбилей: выпускайте кракена
TextRank: 
['животное', 'животный', 'гигантский', 'капитан', 'буйе', 'год', 'длинный', 'длина', 'огромный тушить', 'туше', 'krake', 'kraken', 'существование', 'петлить', 'впоследствии', 'французский', 'тело', 'положить', 'редкий', 'встреча человек', 'свой', 'корвет', 'стать', 'начать', 'кальмар', 'новый', 'спустя восемь', 'цвет', 'цвета', 'английский', 'весь персонаж средневековый', 'научный', 'моряк', 'экипаж', 'волна', 'глубоко', 'глубокий', 'гарпун', 'решить', 'решиться', 'близкий', 'однако', 'легенда', 'создать', 'многие учёный', 'монстр', 'относиться', 'живой природа', 'мир', 'вода', 'вод', 'головоногий', 'щупальце', 'корабль', 'кровожадный', 'француз', 'спустить', 'самый большой', 'хвост', 'схватка', 'видимо', 'полуметровый', 'вложить', 'сила', 'четыре', 'многий', 'способный', 'метр', 'море', 'история', 'оказаться', 'конечность', 'заклинить винт', 'плеск рухнуть', 'труп лошадь', 'сфера мифология', 'спорый', 'спор', 'затем', 'подобный', 'морской', 'труд'

### TF-IDF

In [17]:
X = vectorizer.fit_transform(list(dataframe["lemmatized_text"]))
feature_names = vectorizer.get_feature_names_out()

То, как выделять ключевые слова для текста с помощью TF-IDF, я нашла в интернете. Ссылка на источник: https://github.com/kavgan/nlp-in-practice/blob/master/tf-idf/Keyword%20Extraction%20with%20TF-IDF%20and%20SKlearn.ipynb

In [18]:
def sort_coo(coo_matrix):
    tuples = zip(coo_matrix.col, coo_matrix.data)
    return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)

def extract_topn_from_vector(feature_names, sorted_items, topn=10):
    """get the feature names and tf-idf score of top n items"""
    
    #use only topn items from vector
    sorted_items = sorted_items[:topn]

    score_vals = []
    feature_vals = []

    for idx, score in sorted_items:
        fname = feature_names[idx]
        
        #keep track of feature name and its corresponding score
        score_vals.append(round(score, 3))
        feature_vals.append(feature_names[idx])

    #create a tuples of feature,score
    #results = zip(feature_vals,score_vals)
    results= {}
    for idx in range(len(feature_vals)):
        results[feature_vals[idx]]=score_vals[idx]
    
    return results

def get_keywords(vectorizer, feature_names, texts, keywords_number):
    tf_idf_vector=vectorizer.transform(texts)
    
    results=[]
    for i in range(tf_idf_vector.shape[0]):
    
        # get vector for a single document
        curr_vector=tf_idf_vector[i]
    
        #sort the tf-idf vector by descending order of scores
        sorted_items=sort_coo(curr_vector.tocoo())

        keywords=extract_topn_from_vector(feature_names, sorted_items, keywords_number)
    
        results.append(list(keywords.keys()))
    return results

Насколько я поняла, с помощью TF-IDF можно самостоятельно выбрать количество ключевых слов, которое будет выделено. По-моему, разумно было бы, чтобы оно было примерно таким же, какое получилось у остальных алгоритмов.

Посчитаю среднее количество ключевых слов для всех текстов в алгоритме RAKE

In [19]:
summ = 0
for text in list(dataframe["RAKE_keywords"]):
    summ += len(text)
rake_len = summ / 5

In [20]:
rake_len

11.0

Посчитаю среднее количество ключевых слов для всех текстов в алгоритме TextRank

In [21]:
summ = 0
for text in list(dataframe["TextRank_keywords"]):
    summ += len(text)
textrank_len = summ / 5

In [22]:
textrank_len

73.4

In [23]:
dataframe["TF-IDF_keywords"] = get_keywords(vectorizer, 
                                            feature_names, 
                                            list(dataframe["lemmatized_text"]), 
                                            round((rake_len + textrank_len) / 2))

In [24]:
dataframe

,number,title,text,tags,my_keywords,standart_keywords,standart_keywords_list,lemmatized_text,RAKE_keywords,TextRank_keywords,TF-IDF_keywords
0,1,Юбилей: выпускайте кракена,Ровно 160 лет назад состоялась встреча человек...,"Юбилей, Кальмар, Ноябрь 2021, Лучшее","Монстр, Гигантский кальмар, Глубины моря, Живо...","Монстр, Гигантский кальмар, Глубины моря, Живо...","[монстр, гигантский кальмар, глубина море, жив...",ровно 160 год назад состояться встреча человек...,"[бросить вызов наука, легенда, доказать, наход...","[животное, животный, гигантский, капитан, буйе...","[гигантский, кальмар, буйе, огромный, животное..."
1,2,"Свет, цвет, движение: секреты макросъемки",Читатели журнала и сайта «Вокруг света» наверн...,"Истории, Фотография, Лайфхаки","Макросъемка, Оборудование, Макрофотоохота, При...","Фотография, Макросъемка, Оборудование, Макрофо...","[фотография, макросъемка, оборудование, макроф...",читатель журнал и сайт « вокруг свет » наверня...,"[объект съёмка, объектив, растение, пример, 10...","[фотография, фотограф, макросъемка, большой, ф...","[макросъемка, фотограф, снимка, насекомое, сни..."
2,3,"Мякоть субъективности: 10 тайных символов, скр...",Первая галлюцинация случилась у дали в детстве...,"Сальвадор Дали, Живопись, Искусство, Художник,...","Мягкие часы, Пространство, Время, Вдохновение,...","Сальвадор Дали, Искусство, Художник, Картина, ...","[сальвадор дать, искусство, художник, картина,...",один галлюцинация случиться у дать в детство ....,"[слово нина геташвили, самый знаменитый, худож...","[дать, художник стать, время, символ, час, поэ...","[художник, дать, картина, время, сон, часы, пи..."
3,4,Железное рукопожатие,"Порой небольшие детали — почтовые ящики, дорож...","Истории, Путешествия, Марокко","Декор, Дом, Дверь, Ручка, Талисман, Традиция","Марокко, Декор, Дом, Дверь, Ручка, Талисман, Т...","[марокко, декор, дом, дверь, ручка, талисман, ...","порой небольшой деталь — почтовый ящик , дорож...","[злой дух, пятиконечный звезда, ручка, спасать...","[город, марокко ручка дом, страна, странно, си...","[город, ручка, марокко, декор, звезда, дверь, ..."
4,5,«Помню не зря пятый день ноября»: как недоволь...,В ночь на 5 ноября в Великобритании отмечается...,"Англия, Католицизм, Монархия, Истории","Протестантизм, Католик, Протестант, Король, Па...","Англия, Католицизм, Монархия, Протестантизм, К...","[англия, католицизм, монархия, протестантизм, ...",в ночь на 5 ноябрь в великобритания отмечаться...,"[католический церковь, судьбоносный день, церк...","[свой, год, годами, весь, фокс, фоксом, самый,...","[католик, заговор, король, гай, кейтсби, фокса..."


##### Ключевые слова, предсказанные TF-IDF

In [25]:
for index, text in enumerate(dataframe["TF-IDF_keywords"]):
    print(list(dataframe["title"])[index])
    print("TF-IDF: ")
    print(text)
    print("Standart: ")
    print(list(dataframe["standart_keywords_list"])[index])
    print("\n")

Юбилей: выпускайте кракена
TF-IDF: 
['гигантский', 'кальмар', 'буйе', 'огромный', 'животное', 'тушить', 'существование', 'легенда', 'кракен', 'корабль', 'капитан', 'животный', 'щупальце', 'французский', 'тело', 'положить', 'петлить', 'персонаж', 'палуба', 'наука', 'моряк', 'монстр', 'метр', 'длина', 'головоногий', 'гигантский кальмар', 'гарпун', 'близкий', 'год', 'труд', 'решить', 'относиться', 'оказаться', 'морской', 'впоследствии', 'восемь', 'вода', 'свой', 'весь', 'лишь', 'экипаж', 'шлюпка']
Standart: 
['монстр', 'гигантский кальмар', 'глубина море', 'животное', 'морской легенда', 'миф', 'щупальце', 'кракен', 'кальмар']


Свет, цвет, движение: секреты макросъемки
TF-IDF: 
['макросъемка', 'фотограф', 'снимка', 'насекомое', 'снимать', 'расстояние', 'объектив', 'кадр', 'вспышка', 'фон', 'стоить', 'свет', 'важный', 'штатив', 'фокусный расстояние', 'фокусный', 'съёмка', 'специальный', 'растение', 'проблема', 'получение', 'полезно', 'подробный', 'мм', 'жук', 'герой', 'самый', 'большой', '

### Шаблоны для ключевых слов

По эталонным ключевым словам получилось выделить следующие шаблоны

In [26]:
standart_patterns = [
    ["ADJF"],  # такие слова как "животное" могут восприниматься как прилагательные
    ["NOUN"],
    ["ADJF", "NOUN"],
    ["NOUN", "NOUN"],
    ["NOUN", "INFN"]  # этот шаблон пришлось составить для "сальвадор дали", который стал "сальвадор дать" после лемматизации
]

Функция применения шаблонов, к выделенным ключевым словам

In [27]:
def apply(keywords):
    pattern_keywords = []
    for text in keywords:
        one_text_keywords_list = []
        for item in text:
            help_list = []
            for word in item.split(' '):
                help_list.append(MorphAn.parse(word)[0].tag.POS)
            if help_list in standart_patterns:
                one_text_keywords_list.append(item)
        pattern_keywords.append(one_text_keywords_list)
    return pattern_keywords

In [28]:
dataframe["RAKE_keywords_filtered"] = apply(list(dataframe["RAKE_keywords"]))
dataframe["TextRank_keywords_filtered"] = apply(list(dataframe["TextRank_keywords"]))
dataframe["TF-IDF_keywords_filtered"] = apply(list(dataframe["TF-IDF_keywords"]))

In [29]:
dataframe

,number,title,text,tags,my_keywords,standart_keywords,standart_keywords_list,lemmatized_text,RAKE_keywords,TextRank_keywords,TF-IDF_keywords,RAKE_keywords_filtered,TextRank_keywords_filtered,TF-IDF_keywords_filtered
0,1,Юбилей: выпускайте кракена,Ровно 160 лет назад состоялась встреча человек...,"Юбилей, Кальмар, Ноябрь 2021, Лучшее","Монстр, Гигантский кальмар, Глубины моря, Живо...","Монстр, Гигантский кальмар, Глубины моря, Живо...","[монстр, гигантский кальмар, глубина море, жив...",ровно 160 год назад состояться встреча человек...,"[бросить вызов наука, легенда, доказать, наход...","[животное, животный, гигантский, капитан, буйе...","[гигантский, кальмар, буйе, огромный, животное...","[легенда, вода, существование, кракен]","[животное, животный, гигантский, капитан, буйе...","[гигантский, кальмар, буйе, огромный, животное..."
1,2,"Свет, цвет, движение: секреты макросъемки",Читатели журнала и сайта «Вокруг света» наверн...,"Истории, Фотография, Лайфхаки","Макросъемка, Оборудование, Макрофотоохота, При...","Фотография, Макросъемка, Оборудование, Макрофо...","[фотография, макросъемка, оборудование, макроф...",читатель журнал и сайт « вокруг свет » наверня...,"[объект съёмка, объектив, растение, пример, 10...","[фотография, фотограф, макросъемка, большой, ф...","[макросъемка, фотограф, снимка, насекомое, сни...","[объект съёмка, объектив, растение, пример, ма...","[фотография, фотограф, макросъемка, большой, с...","[макросъемка, фотограф, снимка, насекомое, рас..."
2,3,"Мякоть субъективности: 10 тайных символов, скр...",Первая галлюцинация случилась у дали в детстве...,"Сальвадор Дали, Живопись, Искусство, Художник,...","Мягкие часы, Пространство, Время, Вдохновение,...","Сальвадор Дали, Искусство, Художник, Картина, ...","[сальвадор дать, искусство, художник, картина,...",один галлюцинация случиться у дать в детство ....,"[слово нина геташвили, самый знаменитый, худож...","[дать, художник стать, время, символ, час, поэ...","[художник, дать, картина, время, сон, часы, пи...","[художник, картина, сон, дело, беззащитность]","[художник стать, время, символ, час, муравей, ...","[художник, картина, время, сон, часы, смерть, ..."
3,4,Железное рукопожатие,"Порой небольшие детали — почтовые ящики, дорож...","Истории, Путешествия, Марокко","Декор, Дом, Дверь, Ручка, Талисман, Традиция","Марокко, Декор, Дом, Дверь, Ручка, Талисман, Т...","[марокко, декор, дом, дверь, ручка, талисман, ...","порой небольшой деталь — почтовый ящик , дорож...","[злой дух, пятиконечный звезда, ручка, спасать...","[город, марокко ручка дом, страна, странно, си...","[город, ручка, марокко, декор, звезда, дверь, ...","[злой дух, пятиконечный звезда, ручка, сглаз, ...","[город, страна, декор, бербера, дверь, жизнь, ...","[город, ручка, марокко, декор, звезда, дверь, ..."
4,5,«Помню не зря пятый день ноября»: как недоволь...,В ночь на 5 ноября в Великобритании отмечается...,"Англия, Католицизм, Монархия, Истории","Протестантизм, Католик, Протестант, Король, Па...","Англия, Католицизм, Монархия, Протестантизм, К...","[англия, католицизм, монархия, протестантизм, ...",в ночь на 5 ноябрь в великобритания отмечаться...,"[католический церковь, судьбоносный день, церк...","[свой, год, годами, весь, фокс, фоксом, самый,...","[католик, заговор, король, гай, кейтсби, фокса...","[католический церковь, судьбоносный день, церк...","[свой, год, весь, фокс, фоксом, самый, гай фок...","[католик, заговор, король, гай, кейтсби, фокса..."


### Оценка точности, полноты и F-меры

Функция скоринга

In [30]:
def scoring(standart_keywords, model_keywords):
    precision = []
    recall = []
    for number, text in enumerate(model_keywords):
        count = 0
        for keyword in text:
            if keyword in standart_keywords[number]:
                count += 1
        precision.append(round(count / len(text), 3))
        recall.append(round(count / len(standart_keywords[number]), 3))
    total_precision = round(sum(precision) / len(precision), 3)
    total_recall = round(sum(recall) / len(recall), 3)
    print("Precision (Точность): ", total_precision)
    print("Recall (Полнота): ", total_recall)
    print("F-мера: ", round(2 * total_precision * total_recall / (total_precision + total_recall), 3))

#### Предсказанные ключевые слова без фильтров по шаблонам

In [31]:
print("RAKE: ")
scoring(list(dataframe["standart_keywords_list"]), list(dataframe["RAKE_keywords"]))
print("\n")
print("TextRank: ")
scoring(list(dataframe["standart_keywords_list"]), list(dataframe["TextRank_keywords"]))
print("\n")
print("TF-IDF: ")
scoring(list(dataframe["standart_keywords_list"]), list(dataframe["TF-IDF_keywords"]))

RAKE: 
Precision (Точность):  0.181
Recall (Полнота):  0.215
F-мера:  0.197


TextRank: 
Precision (Точность):  0.059
Recall (Полнота):  0.382
F-мера:  0.102


TF-IDF: 
Precision (Точность):  0.157
Recall (Полнота):  0.686
F-мера:  0.256


#### Предсказанные ключевые слова с фильтрами по шаблонам

In [32]:
print("RAKE: ")
scoring(list(dataframe["standart_keywords_list"]), list(dataframe["RAKE_keywords_filtered"]))
print("\n")
print("TextRank: ")
scoring(list(dataframe["standart_keywords_list"]), list(dataframe["TextRank_keywords_filtered"]))
print("\n")
print("TF-IDF: ")
scoring(list(dataframe["standart_keywords_list"]), list(dataframe["TF-IDF_keywords_filtered"]))

RAKE: 
Precision (Точность):  0.303
Recall (Полнота):  0.215
F-мера:  0.252


TextRank: 
Precision (Точность):  0.085
Recall (Полнота):  0.382
F-мера:  0.139


TF-IDF: 
Precision (Точность):  0.191
Recall (Полнота):  0.686
F-мера:  0.299


#### Вывод


Оценка полноты у ключевых слов с фильтрами и без фильтров не отличается, поскольку формула полноты - пересечение списков ключевых слов, поделенное на количество элементов в эталонной разметке. Наличие или отсутствие фильтров не меняет ничего в этой формуле, поскольку шаблоны для фильтрации создавались так, чтобы покрыть весь эталонный список.


Остальные оценки с появлением фильтров увеличились. Это логично, поскольку точность вычисляется по формуле - пересечение списков ключевых слов, поделенное на количество элементов в разметке, предсказанной алгоритмом. Из-за добавления фильтров размер предсказанной разметки уменьшается, поэтому меняется и значение оценки точности (соответственно увеличивается).


Значение F-меры зависит и от полноты, и от точности. Поскольку увеличилась точность, а полнота не изменилась, увеличилась и F-мера.


В целом, лучшие оценки у алгоритма TF-IDF. Если сравнивать оставшиеся 2 алгоритма, RAKE показывает большую точность, а TextRank - полноту. Возможно, точность RAKE выше, поскольку алгоритм выделяет меньше ключевых слов, соответственно возникает меньше ложно-положительных ответов.

### Проблемы, которые возникли в ходе работы


Одна из трудностей, возникших в процессе выполнения работы, заключается в том, что в случаях с некоторыми текстами алгоритм RAKE не мог выделить ключевые слова, если перед этим из текста были удалены знаки препинания. Я не знаю, с чем связана эта особенность: с какой-то моей ошибкой или же дело в неизвестных мне тонкостях алгоритма. Честно говоря, не могу даже предположить, почему такое может происходить. Поэтому я не стала очищать тексты от имеющихся в них знаков препинания, возможно, это уменьшило точность выделения ключевых слов. Хотя, к сожалению, не могу утверждать это со 100% уверенностью.


Ещё одной проблемой стала неточность в работе морфологического анализатора. Я пользовалась анализатором pymorphy и заметила, что при лемматизации могут возникать некоторые неприятные ошибки, например, в словосочетании "глубины моря" второе слово было воспринято анализатором как глагол, поэтому после лемматизации оно стало выглядеть как "глубины морить", что абсолютно не соответствует изначальному смыслу. Возможно, для исправления подобных ошибок следует попробовать воспользоваться другими морфологическими анализаторами и сравнить результаты их работы. Есть вероятность, что другие алгоритмы справятся с этой задачей лучше.


Также проблема, подобная предыдущей, возникала в тех случаях, когда лемматизировать требовалось имя собственное (не всегда, с некоторыми именами всё было нормально). Так, например, словосочетание "сальвадор дали" после лемматизации стало выглядеть как "сальвадор дать". Для решения этой проблемы возможно следует вручную собрать список имен собственных встречающихся в данном корпусе и в процессе лемматизации просто контролировать, чтобы эти слова пропускались анализатором/оставались такими, какие они есть. Конечно, для большого корпуса и для большого количества данных такое решение нерационально и просто нереализуемо. Однако, к сожалению, я не могу придумать, как можно решить подобную проблему в случаях с большим набором данных.


Ешё одна особенность работы морфологического анализатора, которая могла повлиять на уменьшение точности - не совсем корректное определение частеречного состава ключевых слов, например, слово "животное" не определялось как существительное, поэтому пришлось добавить дополнительный шаблон для него, из-за этого в отфильтрованные списки ключевых слов могло попасть много лишних слов и тем самым уменьшить точность.


Напоследок, можно заметить, что проблема могла возникнуть с иностранными словами, в смысле со словами на английском языке (проблема подобная трудностям с именами собственными). В моем корпусе не было таких случаев, что слово на другом языке можно было бы считать ключевым (в моем корпусе было довольно мало слов на английском), однако, если бы такие случаи были, с выделением подобных ключевых слов точно возникли бы трудности. 